In [1]:
from processing_steps.interpolation import Interpolation
from processing_steps.detect_disturbed_trees import DetectDisturbedTrees
from features.spectral_indices import CalculateIndices
from features.basic_features import BasicFeatures
from features.temporal_features import TemporalFeatures
from processing_steps.data_augmentation import DataAugmentation
from processing_steps.adjust_labels import AdjustLabels
from processing_pipeline import ProcessingPipeline

from processing_steps.aggregation import TimeSeriesAggregate

In [2]:
PATH = "../../../data/preprocessed/testset.csv"

train_steps = [
    BasicFeatures(on=True),
    TimeSeriesAggregate(on=True, freq=2, method="median"),
    CalculateIndices(on=True),
    TemporalFeatures(on=True),
    DataAugmentation(on=False),
    Interpolation(on=True),
    DetectDisturbedTrees(on=False),
    AdjustLabels(on=False),
]

pipeline = ProcessingPipeline(
    path = PATH,
    steps = train_steps
)

df_processed = pipeline.run()
df_processed

,time,id,b2,b3,b4,b5,b6,b7,b8,b8a,...,mcari,reip,masvi2,dvi,month_num,year,season,month_sin,month_cos,date_diff
0,2016-12-26,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,12,2016,Winter,-2.449294e-16,1.000000e+00,NaN
1,2017-01-09,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1,2017,Winter,5.000000e-01,8.660254e-01,14.0
2,2017-01-23,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1,2017,Winter,5.000000e-01,8.660254e-01,14.0
3,2017-02-06,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2,2017,Winter,8.660254e-01,5.000000e-01,14.0
4,2017-02-20,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2,2017,Winter,8.660254e-01,5.000000e-01,14.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
868863,2022-09-12,28201,116.0,241.0,119.0,357.0,1275.0,1616.0,1970.0,1997.0,...,644.400000,722.244009,-0.999493,1851.0,9,2022,Autumn,-1.000000e+00,-1.836970e-16,14.0
868864,2022-09-26,28201,NaN,NaN,120.5,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,9,2022,Autumn,-1.000000e+00,-1.836970e-16,14.0
868865,2022-10-10,28201,128.0,249.0,122.0,331.0,978.0,1131.0,1380.0,1313.0,...,522.545902,718.268934,-0.999276,1258.0,10,2022,Autumn,-8.660254e-01,5.000000e-01,14.0
868866,2022-10-24,28201,NaN,NaN,122.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,10,2022,Autumn,-8.660254e-01,5.000000e-01,14.0
